#### Шаг 1: Инициализация с ограничением памяти

In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import lit
import time

# Даем экзекьютору всего 512 мб памяти
spark = SparkSession.builder \
        .appName("Memory_Lab_Test") \
        .master("local[*]") \
        .config("spark.driver.memory", "512m") \
        .config("spark.executor.memory", "512m") \
        .getOrCreate()

print(f"UI: http://localhost:4040")

UI: http://localhost:4040


#### Шаг 2: Создание и Кэширование (Занимаем Storage)

In [3]:
# Создаем "тяжелый" DataFrame (около 200-300 МБ в памяти)
# Генерируем 5 млн строк с длинной строкой
df = spark.range(0, 5000000).withColumn(
    "dummy_data",
    lit("x" * 100)
)

# Помечаем, что хотим положить его в RAM (Storage Memory)
df.cache()

# Внимание! .cache() - ленивая операция. Данные попадут в память только при Action.
print("Начинаем кэширование...")
start = time.time()
count = df.count() # # Action вызывает чтение и запись в кэш
print(f"Кэшировано {count} строк за {time.time() - start:.2f} сек.")
# ЗАДАНИЕ:
# 1. Идите в Spark UI -> вкладка "Storage".
# 2. Вы должны увидеть таблицу, показывающую наш DataFrame.
# 3. Посмотрите колонки "Size in Memory" и "Fraction Cached".
#    Если Fraction Cached < 100%, значит, всё не влезло!

Начинаем кэширование...
Кэшировано 5000000 строк за 3.09 сек.


#### Шаг 3: Вытеснение (Занимаем Execution)
Теперь попробуем выполнить тяжелую операцию сортировки, которая требует Execution Memory. Поскольку мы заняли место кэшем, Spark придется что-то решать.

In [4]:
# Сортировка требует много памяти для буферов
df.orderBy("id").write.format("noop").mode("overwrite").save()

# ЗАДАНИЕ:
# 1. Идите в Spark UI -> вкладка "Executors".
# 2. Посмотрите на колонки "Storage Memory".
# 3. Вернитесь в "Stages" и посмотрите метрики задач. Были ли Spill?

#### Шаг 4: Освобождение

In [5]:
# Очищаем память
df.unpersist()
# Проверьте вкладку Storage - она должна стать пустой.

DataFrame[id: bigint, dummy_data: string]

In [6]:
spark.stop()